# Movie Recommender System

In [1]:
import pandas as pd
import numpy as np

# ml-1m Dataset Information

- movies.dat - MovieID::Title::Genres
- users.dat  - UserID::Gender::Age::Occupation::Zip-code
- ratings.dat - UserID::MovieID::Rating::Timestamp

In [2]:
# getting movies,users,ratings from different dataset
movies = pd.read_csv('ml-1m/movies.dat',sep='::',header=None,names=['MovieID','Title','Genres'],engine='python',encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat',sep='::',header=None,names=['UserID','Gender','Age','OccupationID','Zip-code'],engine='python',encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat',sep='::',header=None,names=['UserID','MovieID','Rating','Timestamp'],engine='python',encoding='latin-1')

In [3]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users.head()

,UserID,Gender,Age,OccupationID,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# ml-100k Dataset Information

- u.data - full dataset
- u{n}.base and u{n}test are 5 folds for cross validation
- u1.base & u1.test - UserID::MovieID::Rating::Timestamp

In [6]:
training_dataset = pd.read_csv('ml-100k/u1.base', delimiter = '\t',names=['UserID','MovieID','Rating','Timestamp'])
training_dataset.head()

,UserID,MovieID,Rating,Timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [7]:
test_dataset = pd.read_csv('ml-100k/u1.test', delimiter = '\t',names=['UserID','MovieID','Rating','Timestamp'])
test_dataset.head()

,UserID,MovieID,Rating,Timestamp
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


In [8]:
training_set= np.array(training_dataset)
test_set = np.array(test_dataset)
training_set.shape,test_set.shape

((80000, 4), (20000, 4))

# Getting number of Users and Movies in dataset

In [9]:
nb_users= max(max(training_set[:,0]),max(test_set[:,0]))
nb_movies=max(max(training_set[:,1]),max(test_set[:,1]))
nb_users,nb_movies

(943, 1682)

# Structuring the datset


Converting data into matrix
> where `UserID` is `row` and `MovieID` is `column`  
`data[UserID,MovieID-1]=rating`  
- If `User` doesn't have watched the `movie`, rating for that is considered `0` [change later]

`convert` funct takes Input type `data` is `np.array`

In [10]:

def convert(data):
    new_data =[]
    for id_user in range(1,nb_users+1):
        # all movie_id and ratings given by user 
        id_movies = data[:,1][data[:,0]==id_user]
        id_ratings = data[:,2][data[:,0]==id_user]
        #create rating of np.array, contains 0 value of len(nb_movies) 
        ratings = np.zeros(nb_movies)
        #change values of rating where user is given rating else 0         
        ratings[id_movies-1]=id_ratings
        new_data.append(list(ratings))
    return new_data


`convert2` funct takes Input type `data` is `dataframe`

In [11]:
def convert2(data):
    new_data=[]
    for id_user in range(1,nb_users+1):
        # all movie_id and ratings given by user 
        id_movies=data.MovieID[data.UserID ==id_user]
        id_ratings = data.Rating[data.UserID ==id_user]
        #create rating of np.array, contains 0 value of len(nb_movies) 
        ratings = np.zeros(nb_movies)
        #change values of rating where user is given rating else 0 
        ratings[id_movies-1]=id_ratings
        new_data.append(list(ratings))
    return new_data

- `return` type of both `funct` is `nested list` 
- Each `list` in `new_data` contains the `ratings` of all `MovieID` given by `UserID` 

In [12]:
training_set = convert(training_set)
len(training_set),len(training_set[0])

(943, 1682)

In [13]:
test_set = convert(test_set)
len(test_set),len(test_set[0])

(943, 1682)

# `RBM`

## Importing `torch` libraries

In [14]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data

## Convert nested list into `torch` tensors

In [15]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)
training_set.shape, test_set.shape

(torch.Size([943, 1682]), torch.Size([943, 1682]))

## Convert the ratings to binary ratings
> rating = 0 -> -1 [not watched]  
0 < rating > 2 -> 0 [Not liked]  
rating > 3 -> 1 [liked]
 

In [16]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
training_set

tensor([[ 1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1.,  1., -1.,  ..., -1., -1., -1.]])

In [17]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1
test_set

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])

# Architecture of Network

In [18]:
class RBM():
    def __init__(self,nv,nh):
        '''
        initialise weights and bias
        '''

        # weight initialise tensor of size(nh,nv) acc to normal distribution
        self.W=torch.randn(nh,nv)

        # bias for probabilty of hidden nodes given visible nodes, first dim corresponds to batch & second dim to bias
        self.a = torch.randn(1,nh)

        #bias for visible nodes
        self.b = torch.randn(1,nv)
    
    def sample_h(self,x):
        '''
        compute probabilties of hidden nodes given visible nodes
        and after computing prob. we can sample the activations of hidden nodes
        x corresponds to visible neurons(v) in prob p(h) given v
        '''
        #p(h)given v is sigmoid act.fn applied to 
        # (product of w(vector of weights) times x(vector of visiible nodes) + bias of hidden nodes)
        wx= torch.mm(x,self.W.t())
        
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        
        '''
        return prob. and sample of h acc to p_h_given_v
        we are prdicting binary outcomes so, we return bernoulli samples of distribution contains vetor of 0,1.
        0 corresponds to hidden nodes that are not activated after sampling
        1 corresponds to neurons that are activated after sampling
        '''
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self,y):
        '''
        we will compute probabilities of visible nodes given hidden
        and after computing pob. we cam sample the activations of visible nodes
        y corresponds to hidden nodes here
        same as sample_h funct
        '''
        wy = torch.mm(y,self.W)
        activation = wy+self.b.expand_as(wy)
        p_v_given_h= torch.sigmoid(activation)
        return p_v_given_h,torch.bernoulli(p_v_given_h)
    
    def train(self,v0,vk,ph0,phk):
        '''
        k Contrastive Divergence
        approx. log-likelihood gradient
        v0 - input vector containing ratings of user
        vk - visible nodes obtained after k sampling
        ph0 - vector of prob at the 1st iter. first hidden nodes equal 1 given the values of input vector(v0)
        phk - probab of hidden nodes after k sampling given the values of visible nodes(vk)
        
        update tensor of weights W, bias b, bias a.
        '''
        self.W += (torch.mm(v0.t(),ph0)-torch.mm(vk.t(),phk)).t()
        self.b += torch.sum((v0-vk),0)
        self.a += torch.sum((ph0-phk),0)

## RBM Objects init

In [20]:
nv=len(training_set[0])
nh=100          # tunnable
batch_size=100  #tunable
rbm=RBM(nv,nh)

# Training the RBM

In [22]:
nb_epoch=10
for epoch in range(1,nb_epoch+1):
    train_loss=0
    s=0.
    for id_user in range(0,nb_users-batch_size,batch_size):
        
        #visible neurons after k sampling
        vk= training_set[id_user:id_user+batch_size]
        
        #input vector containing user rating
        v0 = training_set[id_user:id_user+batch_size]
        
        ph0 , _ = rbm.sample_h(v0)
        
        #k-step contrastive divergence
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.
  
    print(f"epoch: {epoch} loss: {train_loss/s}")

epoch: 1 loss: 0.24865762889385223
epoch: 2 loss: 0.24826234579086304
epoch: 3 loss: 0.24730004370212555
epoch: 4 loss: 0.24621187150478363
epoch: 5 loss: 0.2484438717365265
epoch: 6 loss: 0.24535591900348663
epoch: 7 loss: 0.24893386662006378
epoch: 8 loss: 0.24655143916606903
epoch: 9 loss: 0.24927014112472534
epoch: 10 loss: 0.24554091691970825


# Testing The RBM

In [23]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print(f'test loss: {test_loss/s}')

test loss: 0.24168653786182404
